imports

pudl set up

In [1]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

import pandas as pd
import sqlalchemy as sa
import pudl

/opt/anaconda3/envs/hub/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)


plotly set up

In [ ]:
# plotly settings
import plotly.express as px
import chart_studio 
import chart_studio.plotly as py
chart_studio.tools.set_credentials_file(username='mcastillo',api_key='3QGy4nzPwGtZmLUypW3U')
chart_studio.tools.set_config_file(world_readable=True,sharing='public')

mapbox set up

In [ ]:
px.set_mapbox_access_token('pk.eyJ1IjoibWNhc3RpbGxvcm1pIiwiYSI6ImNsYWJjcHNhYzA5dTMzc3BpMThpc3RiYmYifQ.xSdph_u4qXYTU163JFOCWQ')

read excel file from online

link: https://screeningtool.geoplatform.gov/en/downloads#3/33.47/-97.5

In [2]:
communities = pd.read_excel('https://static-data-screeningtool.geoplatform.gov/data-versions/1.0/data/score/downloadable/1.0-communities.xlsx')

geoprocessing prep

In [32]:
# get county ID
#make column a string
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].astype(str)
# make sure census tract ID is 11 digits
communities['Census tract 2010 ID'] = communities['Census tract 2010 ID'].str.zfill(11)

#make county fips column
communities['county_id_fips'] = communities['Census tract 2010 ID'].str[0:5]

filter to disadvantaged

In [33]:
disadvantaged_communities = communities.loc[communities['Identified as disadvantaged']==True]

count how many tracts are in each county

In [34]:
num_of_dc = disadvantaged_communities['county_id_fips'].value_counts().reset_index()
num_of_dc = num_of_dc.rename(columns={'index':'county_id_fips','county_id_fips':'num_of_communities'})

connect to total amount

In [36]:
total_dc = communities['county_id_fips'].value_counts().reset_index()
total_dc = total_dc.rename(columns={'index':'county_id_fips','county_id_fips':'total_num_of_communities'})

In [40]:
community_complete = num_of_dc.merge(total_dc,on='county_id_fips',how='left')

In [42]:
community_complete['disadvantaged_percent_of_total_communities'] = community_complete['num_of_communities'] / community_complete['total_num_of_communities'] * 100
community_complete

,county_id_fips,num_of_communities,total_num_of_communities,disadvantaged_percent_of_total_communities
0,06037,1146,2346,48.849105
1,17031,611,1319,46.322972
2,36047,410,761,53.876478
3,48201,402,786,51.145038
4,26163,362,611,59.247136
...,...,...,...,...
2707,31151,1,4,25.000000
2708,31153,1,43,2.325581
2709,31161,1,2,50.000000
2710,31181,1,2,50.000000


get utility service territory from PUDL

In [46]:
eia_service_territory = pudl_out.service_territory_eia861()

In [48]:
eia_service_territory.groupby(['state','utility_id_eia'])

additional request: map of coal plants without retirement date

In [4]:
gens = pudl_out.gens_eia860()
gens_2022 = gens.loc[gens['report_date']=='2022-01-01']

In [23]:
coal_no_retirement = gens_2022[(gens_2022['planned_retirement_date'].isna()) & (gens_2022['fuel_type_code_pudl']=='coal') & (gens_2022['retirement_date'].isna())]

In [25]:
plants_coal_no_retirement = coal_no_retirement.drop_duplicates(subset='plant_id_eia')

In [42]:

fig = px.scatter_mapbox(plants_coal_no_retirement,lat='latitude',lon='longitude',hover_data=['plant_name_eia','utility_name_eia'],size='capacity_mw',title='coal plants without retirement date')

fig.update_layout(
    title_x=0.5,
    title_y=0.95,
    mapbox={'style':'stamen-toner', 'zoom':4.8},
    margin={"l": 0, "r": 0, "b": 0, "t": 80}
)

fig.show()

In [43]:
py.plot(fig,filename='coal-plants-no-retirement',auto_open=True)

'https://plotly.com/~mcastillo/56/'